# Feature importance over time using ExtraTrees. 
We divide the train set into days to be able to see if there is any drastic changes during the 500 days. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier, IsolationForest
from tqdm import tqdm

In [ ]:
train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns}) #limit memory use
train.fillna(train.mean(),inplace=True)
train['action'] = (train['resp'] > 0).astype('int')

features = [col for col in train.columns if 'feature' in col]

In [ ]:
days = set(train['date'].values)

feature_importance = []
for d in tqdm(days):
    clf = ExtraTreesClassifier()
    X = train[train['date']==d][features].values
    y = train[train['date']==d]['action'].values
    clf.fit(X,y)
    feature_importance.append(clf.feature_importances_)

feature_importance = np.asarray(feature_importance)
    

In [ ]:
fig, ax = plt.subplots(figsize=(10, 100))
ax.matshow(feature_importance)

On visual inspection there is only one day that stands out, day 294. This is since this day only has 29 trades. 

In [ ]:
s_days = sorted(zip(days,[len(train[train['date']==d]) for d in days]),key=lambda x: x[1])
print(s_days[0])

### Now let's identify more outlier days.

In [ ]:
pred = IsolationForest().fit_predict(feature_importance)
outlier_days = [day for day,pred in zip(days,pred) if pred < 0]

In [ ]:
outlier_days